---
# Retrieving Climate WEather Station for specific Administrative Regions
---

In this notebook, 

1. We will be retrieving the daily climate data from the ECCC [datamart](https://dd.weather.gc.ca/climate/observations/daily/csv/QC/) for each station over Montréal
2. The downloaded files will be stored in ../data/ECCC/daily/


### Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
import requests
import os
from bs4 import BeautifulSoup
import re

### Define files

In [2]:
stn_inventory = '../data/ECCC/Station_Inventory_EN_AdminRegion.csv'

### Define variables

In [62]:
admin_region = ['Montréal']

data_dir = '../data/ECCC/daily'

datamart = 'https://dd.weather.gc.ca/climate/observations/daily/csv/QC/'

### Read Station Inventory file

In [4]:
stns = pd.read_csv(stn_inventory)
stns

,Name,Province,Climate ID,Station ID,WMO ID,TC ID,Latitude (Decimal Degrees),Longitude (Decimal Degrees),Latitude,Longitude,...,First Year,Last Year,HLY First Year,HLY Last Year,DLY First Year,DLY Last Year,MLY First Year,MLY Last Year,geometry,admin
0,BARRIERE STONEHAM,QUEBEC,7010478,5205,NaN,NaN,47.17,-71.25,471000000,-711500000,...,1963,1976,NaN,NaN,1963.0,1976.0,1963.0,1976.0,POINT (-71.25 47.17),Capitale-Nationale
1,BARRIERE TOURILLI,QUEBEC,7010480,5206,NaN,NaN,47.17,-71.62,471000000,-713700000,...,1949,1960,NaN,NaN,1949.0,1960.0,1949.0,1960.0,POINT (-71.62 47.17),Capitale-Nationale
2,BEAUPORT,QUEBEC,7010565,27803,71578.0,XBO,46.84,-71.20,465013000,-711150000,...,1999,2025,1999.0,2024.0,1999.0,2025.0,NaN,NaN,POINT (-71.2 46.84),Capitale-Nationale
3,BEAUPORT,QUEBEC,7010566,5207,NaN,NaN,46.88,-71.20,465300000,-711200000,...,1982,1985,NaN,NaN,1982.0,1985.0,1982.0,1985.0,POINT (-71.2 46.88),Capitale-Nationale
4,BERTHIERVILLE,QUEBEC,7010720,5208,NaN,NaN,46.05,-73.18,460300000,-731100000,...,1919,1995,NaN,NaN,1919.0,1995.0,1919.0,1995.0,POINT (-73.18 46.05),Lanaudière
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,TASIUJAQ A,QUEBEC,7118366,54019,NaN,YTQ,58.67,-69.96,584004000,-695721000,...,2018,2025,2018.0,2024.0,2018.0,2025.0,NaN,NaN,POINT (-69.96 58.67),Nord-du-Québec
1095,TASIUJAQ A,QUEBEC,7118369,10856,NaN,YTQ,58.67,-69.95,584016000,-695655000,...,1993,2015,1993.0,2015.0,NaN,NaN,NaN,NaN,POINT (-69.95 58.67),Nord-du-Québec
1096,SALLUIT A,QUEBEC,711GG87,10226,NaN,YZG,62.18,-75.67,621055005,-754010001,...,1969,2015,1969.0,2015.0,NaN,NaN,NaN,NaN,POINT (-75.67 62.18),Nord-du-Québec
1097,KANGIQSUALUJJUAQ A,QUEBEC,711L380,10684,NaN,YLU,58.71,-65.99,584227080,-655915020,...,1992,2015,1992.0,2015.0,NaN,NaN,NaN,NaN,POINT (-65.99 58.71),Nord-du-Québec


### Retrieve Climate weather data for stations within administrative region list

In [5]:
# Retrieve page
page = requests.get(datamart)

# Get page content
soup = BeautifulSoup(page.text, 'html.parser')

urls = []

# Retrieve all the CSV files listed on the page
for link in soup.find_all(href=re.compile('.csv$')):    
    urls.append(link.get('href'))

In [55]:
import urllib.request

In [64]:
# Keep only station for desired admin regions
stns_admin = stns[stns['admin'].isin(admin_region)]

# Define query parameters
query_parameters = {"downloadformat": "csv"}

# Loop through all the stations
for idx, stn in stns_admin.iterrows() :

    # Keep Climate Station ID
    stn_id = stn['Climate ID']

    # Retrieve CSV files for the station
    file_list = re.findall(rf"\S+{stn_id}\S+", ' '.join(urls))

    
    # Fetch data 
    for file in file_list :
        # Build URL and path 
        url = os.path.join(datamart, file)
        out = os.path.join(data_dir, file)
        
        # Get URL
        response = requests.get(url, query_parameters)

        # Download file
        if response.ok :
            print(f'Retrieved {file}')
            with open(out, mode="wb") as f_out:
                f_out.write(response.content)
        else :
            print(f'Could not retrieve file {url}')

Retrieved climate_daily_QC_7021945_1960_P1D.csv
Retrieved climate_daily_QC_7021945_1961_P1D.csv
Retrieved climate_daily_QC_7021945_1962_P1D.csv
Retrieved climate_daily_QC_7021945_1963_P1D.csv
Retrieved climate_daily_QC_7024118_1973_P1D.csv
Retrieved climate_daily_QC_7024118_1974_P1D.csv
Retrieved climate_daily_QC_7024118_1975_P1D.csv
Retrieved climate_daily_QC_7024118_1976_P1D.csv
Retrieved climate_daily_QC_7024118_1977_P1D.csv
Retrieved climate_daily_QC_7024118_1978_P1D.csv
Retrieved climate_daily_QC_7024118_1979_P1D.csv
Retrieved climate_daily_QC_7024118_1980_P1D.csv
Retrieved climate_daily_QC_7024118_1981_P1D.csv
Retrieved climate_daily_QC_7024118_1982_P1D.csv
Retrieved climate_daily_QC_7024118_1983_P1D.csv
Retrieved climate_daily_QC_7024256_1965_P1D.csv
Retrieved climate_daily_QC_7024256_1966_P1D.csv
Retrieved climate_daily_QC_7024256_1967_P1D.csv
Retrieved climate_daily_QC_7024256_1968_P1D.csv
Retrieved climate_daily_QC_7024256_1969_P1D.csv
Retrieved climate_daily_QC_7024256_1970_